In [2]:
with open("wizard_of_oz.text","r",encoding="utf-8") as f:
    text = f.read()
    print(text[:200])

﻿DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW Y


In [3]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
print(len(chars))

81


In [13]:
str_to_int = {ch:i for i,ch in enumerate(chars)}
encode = lambda s : [str_to_int[c] for c in s]
encode("Hello")

[32, 58, 65, 65, 68]

In [16]:
int_to_str = {i:ch for i,ch in enumerate(chars)}
decode = lambda list: [int_to_str[i] for i in list]
decode(encode("Hello"))

['H', 'e', 'l', 'l', 'o']